# Draft: MIRI LRS Spectral Extraction

***

## Imports
Describe the libraries we're using here. If there's something unusual, explain what the library is, and why we need it.
- *numpy* to handle array functions
- *astropy.io fits* for accessing FITS files
- *astropy.table Table* for creating tidy tables of the data
- *matplotlib.pyplot* for plotting data

In [ ]:
# For style checking
# %load_ext pycodestyle_magic
# %flake8_on

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

import numpy as np

import astropy.wcs as wcs
from astropy.visualization import simple_norm
from astropy.io import fits
from astropy.table import Table

## Introduction
Extract 1D spectra from 2D MIRI spectral observations.  Show simple "boxcar" extraction as well as a more sophisticated PSF based "optimal" extraction.  PSF based extraction is currently only appropriate for point sources.

Note: Not clear how to use the JWST pipeline `extract_1d` (quite complex) code.

***

## Download File

In [ ]:
from astropy.utils.data import download_file
# backup location
# calfile = "det_image_seq5_MIRIMAGE_P750Lexp1_cal.fits"
calfile_url = "https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/MIRI_LRS_notebook/det_image_seq5_MIRIMAGE_P750Lexp1_cal.fits"
calfile = download_file(calfile_url)

## File information

The downloaded file is a simulation of a LRS slit observation of for the star BD+60d1753, a flux calibration star.  This file was created with MIRISim.
The simulated exposure was reduced using the JWST pipeline (v0.15.0) through the Detector1 and Spec2 stages.

1. (Primary): This HDU contains meta-data related to the observation and data reduction.
2. (SCI): The calibrated image. Units are MJy/sr.
3. (ERR): Uncertainty image.  Units are MJy/sr.
4. (DQ): Data quality image.
5. (VAR_POISSON): Unc. component 1: Poisson uncertainty image.  Units are (MJy/sr)^2.
6. (VAR_RNOISE): Unc. component 2: Read Noise uncertainty image.  Units are (MJy/sr)^2.
7. (VAR_FLAT): Unc. component 3: Flat Field uncertainty image.  Units are (MJy/sr)^2.
8. (ASDF_METADATA): Metadata.

## Loading data

In [ ]:
# load the data
hdu = fits.open(calfile)
# calibrated image in 2nd HDU
data = hdu[1].data

WCS info (not correct - no wavelength info yet)

In [ ]:
# need to know how to get the WCS info for cal file (hint is GWCS in JWST docs)
imwcs = wcs.WCS(hdu[1].header, hdu)
print(imwcs)

Basic information about the image.

In [ ]:
print(hdu[1].data.shape)
print(np.mean(hdu[1].data))
print(np.amin(hdu[1].data), np.amax(hdu[1].data))

Display the full 2D image

In [ ]:
norm_data = simple_norm(data, 'sqrt')
plt.figure(figsize=(6, 6))
plt.imshow(data, norm=norm_data, origin="lower")
plt.title("The full image from the MIRI IMAGER detector")

Display the LRS Slit region

In [ ]:
# python does y, x
# transpose to make it display better
data_lrs_reg = np.transpose(data[145:385, 300:354]) 

norm_data = simple_norm(data_lrs_reg, "sqrt")
plt.figure(figsize=(10, 3))
plt.imshow(data_lrs_reg, norm=norm_data, origin="lower")
plt.title("The full MIRI IMAGE detector image")

## JWST pipeline 1D extraction

In [ ]:
# for reference read in the JWST pipeline extracted spectrum
# currently the flux column is all zeros - so not that useful
# jpipe_x1d = Table.read("det_image_seq5_MIRIMAGE_P750Lexp1_x1d.fits", hdu=1)
# print(jpipe_x1d.columns)
# plot
# fig, ax = plt.subplots(figsize=(6, 6))
# ax.plot(jpipe_x1d['WAVELENGTH'], jpipe_x1d['FLUX'], 'k-', label="boxcar")
# ax.set_title("JWST Pipeline x1d extracted spectrum")
# ax.set_xlabel("pixel")
# ax.set_ylabel("Flux Density [Jy]")
# ax.set_yscale("log")

## Useful Functions

In [ ]:
# useful function that should work for boxcar, boxcar scalled with wavelength
# and [TBD] psf-weighted extractions
import copy

def get_boxcar_weights(center, hwidth, npix):
    """
    Compute the weights given an aperture center, half widths, and number of pixels
    """
    weights = np.zeros((npix))
    # pixels with full weight
    fullpixels = [max(0, int(center - hwidth + 1)),
                  min(int(center + hwidth), npix)]
    weights[fullpixels[0]:fullpixels[1]] = 1.0

    # pixels at the edges of the boxcar with partial weight
    if fullpixels[0] > 0:
        weights[fullpixels[0] - 1] = hwidth - (center - fullpixels[0])
    if fullpixels[1] < npix:
        weights[fullpixels[1]] = hwidth - (fullpixels[1] - center)

    return weights
    
    
def ap_weight_images(center, width, bkg_offset, bkg_width, image_size, waves, wavescale=None):
    """
    Create a weight image that defines the desired extraction aperture
    and the weight image for the requested background regions
   
    Parameters
    ----------
    center : float
        center of aperture in pixels
    width : float
        width of apeture in pixels
    bkg_offset : float
        offset from the extaction edge for the background
        never scaled for wavelength
    bkg_width : float
        width of background region
        never scaled with wavelength
    image_size : tuple with 2 elements
        size of image
    wavescale : float
        scale the width with wavelength (default=None)
        wavescale gives the reference wavelenth for the width value
    
    Returns
    -------
    wimage, bkg_wimage : (2D image, 2D image)
        wimage is the weight image defining the aperature
        bkg_image is the weight image defining the background regions
    """
    wimage = np.zeros(image_size)
    bkg_wimage = np.zeros(image_size)
    yvals = np.arange(image_size[0])
    hwidth = 0.5*width
    cur_bkg_offset = copy.copy(bkg_offset)
    # loop in dispersion direction and compute weights
    for i in range(image_size[1]):
        if wavescale is not None:
            hwidth = 0.5*width*(x[i]/wavescale)

        wimage[:, i] = get_boxcar_weights(center, hwidth, image_size[0])

        # bkg regions
        bkg_wimage[:, i] = get_boxcar_weights(center - hwidth - bkg_offset, bkg_width, image_size[0])
        bkg_wimage[:, i] += get_boxcar_weights(center + hwidth + bkg_offset, bkg_width, image_size[0])

    return (wimage, bkg_wimage)

## Boxcar Extraction

Extract a 1D spectrum using a simple boxcar.  Basically collapse the spectrum in the cross-dispersion direction over a specified number of pixels.

Limitation: not clear how to handle bad pixels with boxcar extraction.  Interpolate?

### Fixed width boxcar

Define extraction parameters

In [ ]:
ext_center = 33
ext_width = 8
bkg_offset = 4
bkg_width = 2

Plot cross-disperion cut showing the extraction parameters

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
y = np.arange(data_lrs_reg.shape[0])
ax.plot(y, data_lrs_reg[:,100], 'k-')
mm = np.array([ext_center, ext_center])
mm_y = ax.get_ylim()
ax.plot(mm, mm_y, 'b--')
ax.plot(mm - ext_width/2., mm_y, 'g:')
ax.plot(mm + ext_width/2., mm_y, 'g:')
ax.set_title("Cross-dispersion Cut at Pixel=100")

Do the extraction

In [ ]:
# x = np.arange(data_lrs_reg.shape[1]) # x in pixels [TBD wavelength instead]
x = np.linspace(14.0, 5.0, num=data_lrs_reg.shape[1])  # approximately correct

# fixed boxcar weight image
wimage_fixedboxcar, wimage_fb_bkg = ap_weight_images(ext_center, ext_width, bkg_offset, bkg_width, data_lrs_reg.shape, x)

norm_data = simple_norm(wimage_fixedboxcar, "sqrt")
plt.figure(figsize=(10, 3))
plt.imshow(wimage_fixedboxcar, norm=norm_data, origin="lower")
plt.title("Fixed boxcar weight image")

norm_data = simple_norm(wimage_fb_bkg, "sqrt")
plt.figure(figsize=(10, 3))
plt.imshow(wimage_fb_bkg, norm=norm_data, origin="lower")
plt.title("Fixed boxcar backgound weight image")

In [ ]:
# extract the spectrum using the weight image
ext1d_boxcar_bkg = np.average(data_lrs_reg, weights=wimage_fb_bkg, axis=0)
data_bkgsub = data_lrs_reg - np.tile(ext1d_boxcar_bkg, (data_lrs_reg.shape[0], 1))

ext1d_boxcar = np.sum(data_lrs_reg*wimage_fixedboxcar, axis=0)
ext1d_boxcar_bkgsub = np.sum(data_bkgsub*wimage_fixedboxcar, axis=0)
# convert from MJy/sr to Jy
ext1d_boxcar *= 1e6*hdu[1].header['PIXAR_SR']
ext1d_boxcar_bkgsub *= 1e6*hdu[1].header['PIXAR_SR']

# plot
fig, ax = plt.subplots(figsize=(6, 6))
gpts = ext1d_boxcar > 0.
ax.plot(x[gpts], ext1d_boxcar[gpts], 'k-', label="boxcar")
ax.plot(x[gpts], ext1d_boxcar_bkgsub[gpts], 'k:', label="boxcar (bkgsub)")
ax.set_title("Fixed boxcar 1D extracted spectrum")
ax.set_xlabel(r"wavelength [$\mu$m] (approx)")
ax.set_ylabel("Flux Density [Jy]")
ax.set_yscale("log")
ax.legend()

### Wavelength scaled width boxcar

Scaling the width of the extraction aperture with wavelength accounts for the changing diffraction limit with wavelength.

TBD: Trace the peak versus wavelength to allow for a spectral tilt.

In [ ]:
# boxcar scaled with wavelength
wimage_scaledboxcar, wimage_sb_bkg = ap_weight_images(ext_center, ext_width, bkg_offset, bkg_width, data_lrs_reg.shape, x, wavescale=10.0)

norm_data = simple_norm(wimage_scaledboxcar, "sqrt")
plt.figure(figsize=(10, 3))
plt.imshow(wimage_scaledboxcar, norm=norm_data, origin="lower")
plt.title("Scaled boxcar weight image")

norm_data = simple_norm(wimage_sb_bkg, "sqrt")
plt.figure(figsize=(10, 3))
plt.imshow(wimage_sb_bkg, norm=norm_data, origin="lower")
plt.title("Scaled boxcar backgound weight image")

In [ ]:
# extract the spectrum using the weight image
ext1d_boxcar_bkg = np.average(data_lrs_reg, weights=wimage_sb_bkg, axis=0)
data_bkgsub = data_lrs_reg - np.tile(ext1d_boxcar_bkg, (data_lrs_reg.shape[0], 1))

ext1d_scaledboxcar = np.sum(data_lrs_reg*wimage_scaledboxcar, axis=0)
ext1d_scaledboxcar_bkgsub = np.sum(data_bkgsub*wimage_scaledboxcar, axis=0)
# convert from MJy/sr to Jy
ext1d_scaledboxcar *= 1e6*hdu[1].header['PIXAR_SR']
ext1d_scaledboxcar_bkgsub *= 1e6*hdu[1].header['PIXAR_SR']

# plot
fig, ax = plt.subplots(figsize=(6, 6))
gpts = ext1d_boxcar_bkgsub > 0.
ax.plot(x[gpts], ext1d_boxcar_bkgsub[gpts], 'k:', label="fixed boxcar (bkgsub)")
gpts = ext1d_scaledboxcar_bkgsub > 0.
ax.plot(x[gpts], ext1d_scaledboxcar_bkgsub[gpts], 'k-', label="scaled boxcar (bkgsub)")
ax.set_title("Scaled boxcar 1D extracted spectrum")
ax.set_xlabel("wavelength (approx)")
ax.set_ylabel("Flux Density [Jy]")
ax.set_yscale("log")
ax.legend()


## PSF based Extraction

Use the knowledge of the PSF and how it changes with wavelength to provide an improved extration.

In [ ]:
# TBD
# read in a reference file that gives the spatial profile as a function of wavelength (pixel intially)
# fit the spatial profile plus a constant/line for the background for each wavelength
#   fitted scaling factor gives the flux density at that pixel
# use astropy.modeling model sets to fit simultaneously versus wavelength

## Additional Resources
Provide links to appropriate JDox pages for MIRI LRS and JWST pipeline.

- [MIRI LRS](https://jwst-docs.stsci.edu/mid-infrared-instrument/miri-observing-modes/miri-low-resolution-spectroscopy)
- [MIRISim](http://www.stsci.edu/jwst/science-planning/proposal-planning-toolbox/mirisim)
- [JWST pipeline](https://jwst-docs.stsci.edu/jwst-data-reduction-pipeline)
- Include a reference for PSF based extraction (Horne?).

## About this notebook

**Author:** Karl Gordon, JWST
**Updated On:** 2020-04-22

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 